# Compilation benchmark

> Functions to test and benchmark unitary compilation.

In [ ]:
#| default_exp benchmark.bench_compilation

In [ ]:
#| export
from genQC.imports import *

## Special unitaries

#### Quantum Fourier transform

$$
\begin{equation}
   \mathrm{QFT}: |x\rangle \mapsto \frac{1}{\sqrt{N}} \sum_{k=0}^{N-1}  \omega_N^{xk}\;|k\rangle,
\end{equation}
$$
where
$$
\begin{equation}
    \omega_N=\exp{\frac{2\pi i}{N}} \quad\text{and}\quad N=2^{\text{qubits}}.
\end{equation}
$$

In [ ]:
#| export
class SpecialUnitaries:
    """Special unitary matrices to benchmark compilation."""
    
    @staticmethod
    def QFT(num_qubits: int) -> torch.Tensor:
        """The Quantum Fourier transform (QFT) unitary for `num_qubits`-qubits."""
        
        N  = 2**num_qubits
        wN = np.exp(2.0j*np.pi/N)

        U = torch.zeros((N, N), dtype=torch.complex128)      
        for x in range(N):
            U[:, x] = torch.tensor([np.power(wN, x*k, dtype=complex) for k in range(N)])

        U *= 1.0/np.sqrt(N)   
        return U

In [ ]:
# test QFT for N=4
QFT_2_qubits = 0.5 * torch.tensor([[1,  1,   1,  1],
                                   [1,  1j, -1, -1j],
                                   [1, -1,   1, -1],
                                   [1, -1j, -1,  1j]], dtype=torch.complex128)

assert torch.allclose(SpecialUnitaries.QFT(num_qubits=2), QFT_2_qubits)

In [ ]:
np.round(SpecialUnitaries.QFT(3), 3)

tensor([[ 0.3540+0.0000j,  0.3540+0.0000j,  0.3540+0.0000j,  0.3540+0.0000j,  0.3540+0.0000j,  0.3540+0.0000j,  0.3540+0.0000j,  0.3540+0.0000j],
        [ 0.3540+0.0000j,  0.2500+0.2500j,  0.0000+0.3540j, -0.2500+0.2500j, -0.3540+0.0000j, -0.2500-0.2500j,  0.0000-0.3540j,  0.2500-0.2500j],
        [ 0.3540+0.0000j,  0.0000+0.3540j, -0.3540+0.0000j,  0.0000-0.3540j,  0.3540+0.0000j,  0.0000+0.3540j, -0.3540+0.0000j,  0.0000-0.3540j],
        [ 0.3540+0.0000j, -0.2500+0.2500j,  0.0000-0.3540j,  0.2500+0.2500j, -0.3540+0.0000j,  0.2500-0.2500j,  0.0000+0.3540j, -0.2500-0.2500j],
        [ 0.3540+0.0000j, -0.3540+0.0000j,  0.3540+0.0000j, -0.3540+0.0000j,  0.3540+0.0000j, -0.3540+0.0000j,  0.3540+0.0000j, -0.3540+0.0000j],
        [ 0.3540+0.0000j, -0.2500-0.2500j,  0.0000+0.3540j,  0.2500-0.2500j, -0.3540+0.0000j,  0.2500+0.2500j,  0.0000-0.3540j, -0.2500+0.2500j],
        [ 0.3540+0.0000j,  0.0000-0.3540j, -0.3540+0.0000j,  0.0000+0.3540j,  0.3540+0.0000j,  0.0000-0.3540j, -0.3540+0.000

## Hamiltonian evolutions

In [ ]:
#| export
sigma_x = torch.tensor([[0, 1],
                        [1, 0]],
                       dtype=torch.complex128)

sigma_y = torch.tensor([[ 0, -1j],
                        [1j,   0]],
                       dtype=torch.complex128)

sigma_z = torch.tensor([[1,  0],
                        [0, -1]],
                       dtype=torch.complex128)

In [ ]:
assert torch.allclose(sigma_x@sigma_x, torch.eye(2, dtype=torch.complex128))
assert torch.allclose(sigma_y@sigma_y, torch.eye(2, dtype=torch.complex128))
assert torch.allclose(sigma_z@sigma_z, torch.eye(2, dtype=torch.complex128))

In [ ]:
#| export
def qubit_tensor_product(num_qubits: int, *ops: torch.Tensor, pos: int | Sequence[int]) -> torch.Tensor:
    """
    Make tensor product with identities, assumes `ops` placed at `pos` in the tensor product ordering.
    """

    _ops = [torch.eye(2) for i in range(num_qubits)]

    if isinstance(pos, int):
        pos = [pos]
    elif isinstance(pos, Sequence):
        assert len(pos) == len(ops)
    else:
        raise NotImplementedError()

    for pos_i, ops_i in zip(pos, ops):
        _ops[pos_i] = ops_i
        
    mat = _ops[0]
    for op in _ops[1:]:
        mat = torch.kron(mat, op)

    return mat

$\sigma_x \otimes I$:

In [ ]:
qubit_tensor_product(2, sigma_x, pos=0)

tensor([[0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
        [1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j]], dtype=torch.complex128)

$I \otimes \sigma_x$:

In [ ]:
qubit_tensor_product(2, sigma_x, pos=-1)

tensor([[0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
        [1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
        [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j]], dtype=torch.complex128)

$\sigma_z \otimes \sigma_z$:

In [ ]:
qubit_tensor_product(2, sigma_z, sigma_z, pos=[0, 1])

tensor([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j, -1.+0.j,  0.+0.j, -0.+0.j],
        [ 0.+0.j,  0.+0.j, -1.+0.j, -0.+0.j],
        [ 0.+0.j, -0.+0.j, -0.+0.j,  1.-0.j]], dtype=torch.complex128)

#### Base Hamiltonian

In [ ]:
#| export
class BaseHamiltonian(abc.ABC):
    """Base implementation of a Hamiltonian."""

    def __init__(self, device: Optional[str | torch.device] = None) -> None:
        self.device = default(device, "cpu")
        self._generate_matrix()
        
        if not torch.allclose(self.data.adjoint(), self.data):
            raise RuntimeError("Generated Hamiltonian matrix is not self-adjoint!")
    
    @abc.abstractmethod
    def _generate_matrix(self) -> torch.Tensor:
        """Generates the Hamiltonian matrix into `self.data`."""
        raise NotImplementedError()

    def get_evolution(self, t: float | torch.Tensor, split_complex_channel: bool = False, dtype: Optional[torch.dtype] = None) -> torch.Tensor:
        """
        Assuming `h_bar=1`. Returns the unitary evolution in marix form.
        """
        U = torch.linalg.matrix_exp(-1j * t * self.data)

        if split_complex_channel:
            U = torch.stack([torch.real(U), torch.imag(U)])

        if exists(dtype):
            U = U.to(dtype)
        
        return U

#### Ising Hamiltonian

Defined as
$$
H = -J \sum_{\langle i, j \rangle} \sigma_i^z \sigma_j^z - h \sum_i \sigma_i^x,
$$
where $J$ is the coupling constant and $h$ a magnetic field.

In [ ]:
#| export
class IsingHamiltonian(BaseHamiltonian):
    """Implementation of the Ising Hamiltonian on a qubit chain."""
    
    def __init__(self, 
                 h: float, 
                 J: float, 
                 num_qubits: int, 
                 periodic_boundary: bool = True,
                 device: Optional[str | torch.device] = None) -> None:
        """
        h:     Magnetic field 
        J:     Coupling constant 
        """
        self.h = h
        self.J = J      
        self.num_qubits = num_qubits
        self.periodic_boundary = periodic_boundary
        super().__init__(device)
        
    def _generate_matrix(self) -> torch.Tensor:
        """
        Note:  We take big endian convention in placing the `i,j`-sigmas in tensor product ordering.
               For little endian we need to use `pos = self.num_qubits-i`.
        """
        
        N   = 2**self.num_qubits
        ham = torch.zeros((N, N), dtype=torch.complex128)

        pairs = [(i, i+1) for i in range(self.num_qubits-1)]
        
        if self.periodic_boundary:
            pairs.append((self.num_qubits-1, 0))

        for (i, j) in pairs:
            Z_term = qubit_tensor_product(self.num_qubits, sigma_z, sigma_z, pos=[i, j])

            # Coupling + Perturbation
            ham += -self.J * Z_term

        # Magnetic
        for i in range(self.num_qubits):
            ham += -self.h * qubit_tensor_product(self.num_qubits, sigma_x, pos=i)

        self.data = ham.to(self.device)

In [ ]:
hamiltonian = IsingHamiltonian(h=0, J=1, num_qubits=2)
hamiltonian.data

tensor([[-2.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  2.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  2.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  0.+0.j, -2.+0.j]], dtype=torch.complex128)

Eigenvalues of this Hamiltonian:

In [ ]:
torch.linalg.eigvalsh(hamiltonian.data)

tensor([-2., -2.,  2.,  2.], dtype=torch.float64)

In [ ]:
e, v = torch.linalg.eigh(hamiltonian.data)
v

tensor([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
        [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j]], dtype=torch.complex128)

And the evolution unitary is:

In [ ]:
hamiltonian.get_evolution(t=np.pi/6)

tensor([[0.5000+0.8660j, 0.0000+0.0000j, 0.0000+0.0000j, 0.0000+0.0000j],
        [0.0000+0.0000j, 0.5000-0.8660j, 0.0000+0.0000j, 0.0000+0.0000j],
        [0.0000+0.0000j, 0.0000+0.0000j, 0.5000-0.8660j, 0.0000+0.0000j],
        [0.0000+0.0000j, 0.0000+0.0000j, 0.0000+0.0000j, 0.5000+0.8660j]], dtype=torch.complex128)

#### XXZ Hamiltonian

Defined as
$$
H = -J \sum_{\langle i, j \rangle} ( \sigma_i^x \sigma_j^x + \sigma_i^y \sigma_j^y + \Delta \sigma_i^z \sigma_j^z ) - h \sum_i \sigma_i^x,
$$
where $J$ is the coupling constant, $\Delta$ a perturbation and $h$ a magnetic field.

In [ ]:
#| export
class XXZHamiltonian(BaseHamiltonian):
    """Implementation of the XXZ Hamiltonian on a qubit chain."""
    
    def __init__(self, 
                 h: float, 
                 J: float, 
                 delta: float, 
                 num_qubits: int, 
                 periodic_boundary: bool = True,
                 device: Optional[str | torch.device] = None) -> None:
        """
        h:     Magnetic field 
        J:     Coupling constant 
        delta: Perturbation
        """
        self.h = h
        self.J = J      
        self.delta = delta
        self.num_qubits = num_qubits
        self.periodic_boundary = periodic_boundary
        super().__init__(device)
        
    def _generate_matrix(self) -> torch.Tensor:
        """
        Note:  We take big endian convention in placing the `i,j`-sigmas in tensor product ordering.
               For little endian we need to use `pos = self.num_qubits-i`.
        """
        
        N   = 2**self.num_qubits
        ham = torch.zeros((N, N), dtype=torch.complex128)

        pairs = [(i, i+1) for i in range(self.num_qubits-1)]
        
        if self.periodic_boundary:
            pairs.append((self.num_qubits-1, 0))

        for (i, j) in pairs:
            X_term = qubit_tensor_product(self.num_qubits, sigma_x, sigma_x, pos=[i, j])
            Y_term = qubit_tensor_product(self.num_qubits, sigma_y, sigma_y, pos=[i, j])
            Z_term = qubit_tensor_product(self.num_qubits, sigma_z, sigma_z, pos=[i, j])

            # Coupling + Perturbation
            ham += -self.J * (X_term + Y_term + self.delta * Z_term)

        # Magnetic
        for i in range(self.num_qubits):
            ham += -self.h * qubit_tensor_product(self.num_qubits, sigma_x, pos=i)

        self.data = ham.to(self.device)

In [ ]:
hamiltonian = XXZHamiltonian(h=1, J=1, delta=1, num_qubits=2)
hamiltonian.data

tensor([[-2.+0.j, -1.+0.j, -1.+0.j,  0.+0.j],
        [-1.+0.j,  2.+0.j, -4.+0.j, -1.+0.j],
        [-1.+0.j, -4.+0.j,  2.+0.j, -1.+0.j],
        [ 0.+0.j, -1.+0.j, -1.+0.j, -2.+0.j]], dtype=torch.complex128)

Eigenvalues of this Hamiltonian:

In [ ]:
torch.linalg.eigvalsh(hamiltonian.data)

tensor([-4.0000e+00, -2.0000e+00,  8.8818e-16,  6.0000e+00], dtype=torch.float64)

And the evolution unitary is:

In [ ]:
hamiltonian.get_evolution(t=np.pi/6)

tensor([[ 0.3750+0.6495j, -0.3750+0.2165j, -0.3750+0.2165j, -0.1250-0.2165j],
        [-0.3750+0.2165j, -0.3750+0.2165j,  0.6250+0.2165j, -0.3750+0.2165j],
        [-0.3750+0.2165j,  0.6250+0.2165j, -0.3750+0.2165j, -0.3750+0.2165j],
        [-0.1250-0.2165j, -0.3750+0.2165j, -0.3750+0.2165j,  0.3750+0.6495j]], dtype=torch.complex128)

In [ ]:
hamiltonian.get_evolution(t=np.pi/6, split_complex_channel=True)

tensor([[[ 0.3750, -0.3750, -0.3750, -0.1250],
         [-0.3750, -0.3750,  0.6250, -0.3750],
         [-0.3750,  0.6250, -0.3750, -0.3750],
         [-0.1250, -0.3750, -0.3750,  0.3750]],

        [[ 0.6495,  0.2165,  0.2165, -0.2165],
         [ 0.2165,  0.2165,  0.2165,  0.2165],
         [ 0.2165,  0.2165,  0.2165,  0.2165],
         [-0.2165,  0.2165,  0.2165,  0.6495]]], dtype=torch.float64)

# Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()